In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense, Conv2D, LSTM, CuDNNLSTM, Flatten, Dropout, Reshape, BatchNormalization, PReLU, ELU
from keras import optimizers
from keras import initializers
from keras.utils import to_categorical

In [3]:
def slidingWindow(sequence, labels, winSize, step, noNull):

    # Verify the inputs
    try: it = iter(sequence)
    except TypeError:
        raise Exception("**ERROR** sequence must be iterable.")
    if not ((type(winSize) == type(0)) and (type(step) == type(0))):
        raise Exception("**ERROR** type(winSize) and type(step) must be int.")
    if step > winSize:
        raise Exception("**ERROR** step must not be larger than winSize.")
    if winSize > len(sequence):
        raise Exception("**ERROR** winSize must not be larger than sequence length.")
 
    # number of chunks
    numOfChunks = ((len(sequence)-winSize)//step)+1
 
    # Do the work
    for i in range(0,numOfChunks*step,step):
        segment = sequence[i:i+winSize]
        seg_labels = labels[i:i+winSize]
        if noNull:
            if seg_labels[-1] != 0:
                yield segment, seg_labels
        else:
            yield segment, seg_labels

In [4]:
def segment_data(X_train, y_train, X_val, y_val, X_test, y_test, winSize, step, noNull=False):
    assert len(X_train) == len(y_train)
    assert len(X_val) == len(y_val)
    assert len(X_test) == len(y_test)
    # obtain chunks of data
    train_chunks = slidingWindow(X_train, y_train , winSize, step, noNull)
    val_chunks = slidingWindow(X_val, y_val, winSize, step, noNull)
    test_chunks = slidingWindow(X_test, y_test, winSize, step, noNull)
    
    # segment the data
    train_segments = []
    train_labels = []
    for chunk in train_chunks:
        data = chunk[0]
        labels = chunk[1]
        train_segments.append(data)
        train_labels.append(labels[-1])
        
    val_segments = []
    val_labels = []
    for chunk in val_chunks:
        data = chunk[0]
        labels = chunk[1]
        val_segments.append(data)
        val_labels.append(labels[-1])
    
    test_segments = []
    test_labels = []
    for chunk in test_chunks:
        data = chunk[0]
        labels = chunk[1]
        test_segments.append(data)
        test_labels.append(labels[-1])
        
    return np.array(train_segments), np.array(train_labels), np.array(val_segments), np.array(val_labels), np.array(test_segments), np.array(test_labels)

In [5]:
def prepare_data(train_data, val_data, test_data):
    encoder = OneHotEncoder()
    train_labels = encoder.fit_transform(train_data['labels'].values.reshape(-1,1)).toarray()
    val_labels = encoder.transform(val_data['labels'].values.reshape(-1,1)).toarray()
    test_labels = encoder.transform(test_data['labels'].values.reshape(-1,1)).toarray()
    scaler = MinMaxScaler(feature_range=(0, 1))
    train_data.drop(['labels'], axis=1, inplace=True)
    val_data.drop(['labels'], axis=1, inplace=True)
    test_data.drop(['labels'], axis=1, inplace=True)
    data = pd.concat([train_data,val_data,test_data])
    scaler.fit(data)
    train_data = scaler.transform(train_data)
    val_data = scaler.transform(val_data)
    test_data = scaler.transform(test_data)
    
    return train_data, val_data, test_data, train_labels, val_labels, test_labels

In [6]:
# import train data
adl_1_1 = pd.read_csv("./reduced_dataset/ADL1Opportunity_taskB2_S1.csv",header=None)
adl_1_2 = pd.read_csv("./reduced_dataset/ADL2Opportunity_taskB2_S1.csv",header=None)
adl_1_3 = pd.read_csv("./reduced_dataset/ADL3Opportunity_taskB2_S1.csv",header=None)
adl_1_4 = pd.read_csv("./reduced_dataset/ADL4Opportunity_taskB2_S1.csv",header=None)
adl_1_5 = pd.read_csv("./reduced_dataset/ADL5Opportunity_taskB2_S1.csv",header=None)
drill_1 = pd.read_csv("./reduced_dataset/Drill1Opportunity_taskB2.csv",header=None)
adl_2_1 = pd.read_csv("./reduced_dataset/ADL1Opportunity_taskB2_S2.csv",header=None)
adl_2_2 = pd.read_csv("./reduced_dataset/ADL2Opportunity_taskB2_S2.csv",header=None)
drill_2 = pd.read_csv("./reduced_dataset/Drill2Opportunity_taskB2.csv",header=None)
adl_3_1 = pd.read_csv("./reduced_dataset/ADL1Opportunity_taskB2_S3.csv",header=None)
adl_3_2 = pd.read_csv("./reduced_dataset/ADL2Opportunity_taskB2_S3.csv",header=None)
drill_3 = pd.read_csv("./reduced_dataset/Drill3Opportunity_taskB2.csv",header=None)

# import validation data
adl_2_3 = pd.read_csv("./reduced_dataset/ADL3Opportunity_taskB2_S2.csv",header=None)
adl_3_3 = pd.read_csv("./reduced_dataset/ADL3Opportunity_taskB2_S3.csv",header=None)

# import test data
adl_2_4 = pd.read_csv("./reduced_dataset/ADL4Opportunity_taskB2_S2.csv",header=None)
adl_2_5 = pd.read_csv("./reduced_dataset/ADL5Opportunity_taskB2_S2.csv",header=None)
adl_3_4 = pd.read_csv("./reduced_dataset/ADL4Opportunity_taskB2_S3.csv",header=None)
adl_3_5 = pd.read_csv("./reduced_dataset/ADL5Opportunity_taskB2_S3.csv",header=None)

In [7]:
train_frames = [adl_1_1,adl_1_2,adl_1_3,adl_1_4,adl_1_5,drill_1,adl_2_1,adl_2_2,drill_2,adl_3_1,adl_3_2,drill_3]
val_frames = [adl_2_3,adl_3_3]
test_frames = [adl_2_4,adl_2_5,adl_3_4,adl_3_5]
train_data = pd.concat(train_frames)
val_data = pd.concat(val_frames)
test_data = pd.concat(test_frames)
train_data.rename(columns ={21: 'labels'}, inplace =True)
val_data.rename(columns ={21: 'labels'}, inplace =True)
test_data.rename(columns ={21: 'labels'}, inplace =True)

In [8]:
print("shapes: train {0}, val {1}, test {2}".format(train_data.shape, val_data.shape, test_data.shape))

shapes: train (497014, 22), val (60949, 22), test (118750, 22)


In [9]:
scaled_train, scaled_val, scaled_test, train_labels, val_labels, test_labels = prepare_data(train_data, val_data, test_data)

In [10]:
print(scaled_train.shape)
print(train_labels.shape)

(497014, 21)
(497014, 18)


In [11]:
num_sensors = 21
window_size = 24
step_size = 12
classes = 18

In [12]:
train_segments, train_labels, val_segments, val_labels, test_segments, test_labels = segment_data(scaled_train, train_labels, scaled_val, val_labels,
                                                                                                  scaled_test, test_labels, window_size, step_size)

In [13]:
# reshape input for CNN
reshaped_train = train_segments.reshape(-1, window_size, num_sensors, 1)
reshaped_val = val_segments.reshape(-1, window_size, num_sensors, 1)
reshaped_test = test_segments.reshape(-1, window_size, num_sensors, 1)

In [61]:
model = Sequential()

size_of_kernel = (5,1)
kernel_strides = 1
num_filters = 64
num_lstm_cells = 128
dropout_prob = 0.5
inputshape = (window_size, num_sensors, 1)

In [62]:
model.add(BatchNormalization(input_shape=inputshape))

In [63]:
model.add(Conv2D(num_filters, kernel_size=size_of_kernel, strides=kernel_strides,
                 kernel_initializer='glorot_normal', name='1_conv_layer'))
model.add(PReLU())

In [64]:
model.add(Conv2D(num_filters, kernel_size=size_of_kernel, strides=kernel_strides,
                 kernel_initializer='glorot_normal',name='2_conv_layer'))
model.add(PReLU())

In [65]:
model.add(Conv2D(num_filters, kernel_size=size_of_kernel, strides=kernel_strides,
                 kernel_initializer='glorot_normal', name='3_conv_layer'))
model.add(PReLU())

In [66]:
model.add(Conv2D(num_filters, kernel_size=size_of_kernel, strides=kernel_strides,
                 kernel_initializer='glorot_normal', name='4_conv_layer'))
model.add(PReLU())

In [67]:
model.add(Reshape((8, num_filters*num_sensors)))

In [68]:
model.add(CuDNNLSTM(num_lstm_cells,kernel_initializer='glorot_normal', return_sequences=True, name='1_lstm_layer'))

model.add(Dropout(dropout_prob, name='1_dropout_layer'))

model.add(CuDNNLSTM(num_lstm_cells,kernel_initializer='glorot_normal',return_sequences=False, name='2_lstm_layer'))

model.add(Dropout(dropout_prob, name='2_dropout_layer'))

model.add(Dense(64,kernel_initializer='glorot_normal',
                bias_initializer=initializers.Constant(value=0.1), name='dense2_layer'))

model.add(Dropout(dropout_prob, name='3_dropout_layer'))

model.add(Dense(classes,kernel_initializer='glorot_normal',
                bias_initializer=initializers.Constant(value=0.1), activation='softmax', name='softmax_layer'))

opt = optimizers.Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [69]:
for layer in model.layers:
    print(str(layer.name) + ': input shape: ' + str(layer.input_shape) + ' output shape: ' + str(layer.output_shape))

batch_normalization_6: input shape: (None, 24, 21, 1) output shape: (None, 24, 21, 1)
1_conv_layer: input shape: (None, 24, 21, 1) output shape: (None, 20, 21, 64)
2_conv_layer: input shape: (None, 20, 21, 64) output shape: (None, 16, 21, 64)
3_conv_layer: input shape: (None, 16, 21, 64) output shape: (None, 12, 21, 64)
4_conv_layer: input shape: (None, 12, 21, 64) output shape: (None, 8, 21, 64)
reshape_6: input shape: (None, 8, 21, 64) output shape: (None, 8, 1344)
1_lstm_layer: input shape: (None, 8, 1344) output shape: (None, 8, 128)
2_dropout_layer: input shape: (None, 8, 128) output shape: (None, 8, 128)
2_lstm_layer: input shape: (None, 8, 128) output shape: (None, 128)
3_dropout_layer: input shape: (None, 128) output shape: (None, 128)
dense2_layer: input shape: (None, 128) output shape: (None, 32)
4_dropout_layer: input shape: (None, 32) output shape: (None, 32)
softmax_layer: input shape: (None, 32) output shape: (None, 18)


In [71]:
batchSize = 200
train_epoches = 50

model.fit(reshaped_train,train_labels,validation_data=(reshaped_val,val_labels),epochs=train_epoches,batch_size=batchSize,verbose=1)

print('Calculating score.. ')
score = model.evaluate(reshaped_test,test_labels,verbose=1)
print(score)
model.save('taskB2_all_Subjects_CNN_LSTM_our_elaboration_reduced.h5')

Train on 41416 samples, validate on 5078 samples
Epoch 1/50
41416/41416 [==============================] - 12s 278us/step - loss: 0.3269 - acc: 0.8774 - val_loss: 0.5705 - val_acc: 0.8551
Epoch 2/50
41416/41416 [==============================] - 12s 278us/step - loss: 0.3209 - acc: 0.8810 - val_loss: 0.6011 - val_acc: 0.8511
Epoch 3/50
41416/41416 [==============================] - 12s 280us/step - loss: 0.3176 - acc: 0.8804 - val_loss: 0.5815 - val_acc: 0.8539
Epoch 4/50
41416/41416 [==============================] - 12s 279us/step - loss: 0.3122 - acc: 0.8823 - val_loss: 0.6084 - val_acc: 0.8604
Epoch 5/50
41416/41416 [==============================] - 12s 280us/step - loss: 0.3108 - acc: 0.8840 - val_loss: 0.6122 - val_acc: 0.8616
Epoch 6/50
41416/41416 [==============================] - 12s 279us/step - loss: 0.3105 - acc: 0.8837 - val_loss: 0.6250 - val_acc: 0.8523
Epoch 7/50
41416/41416 [==============================] - 12s 281us/step - loss: 0.2969 - acc: 0.8874 - val_loss: 0.5

In [72]:
train_epoches = 100
all_train = np.concatenate((reshaped_train, reshaped_val))
all_labels = np.concatenate((train_labels, val_labels))
model.fit(all_train,all_labels,validation_data=(reshaped_test,test_labels),epochs=train_epoches,batch_size=batchSize,verbose=1)

print('Calculating score.. ')
score = model.evaluate(reshaped_test,test_labels,verbose=1)
print(score)
model.save('taskB2_all_Subjects_CNN_LSTM_our_elaboration_reduced.h5')

Train on 46494 samples, validate on 9894 samples
Epoch 1/100
46494/46494 [==============================] - 13s 286us/step - loss: 0.2252 - acc: 0.9322 - val_loss: 0.5973 - val_acc: 0.8672
Epoch 2/100
46494/46494 [==============================] - 13s 281us/step - loss: 0.2034 - acc: 0.9375 - val_loss: 0.5677 - val_acc: 0.8728
Epoch 3/100
46494/46494 [==============================] - 13s 281us/step - loss: 0.1917 - acc: 0.9410 - val_loss: 0.5823 - val_acc: 0.8664
Epoch 4/100
46494/46494 [==============================] - 13s 281us/step - loss: 0.1835 - acc: 0.9430 - val_loss: 0.5860 - val_acc: 0.8794
Epoch 5/100
46494/46494 [==============================] - 13s 281us/step - loss: 0.1782 - acc: 0.9446 - val_loss: 0.6281 - val_acc: 0.8635
Epoch 6/100
46494/46494 [==============================] - 13s 283us/step - loss: 0.1799 - acc: 0.9448 - val_loss: 0.5947 - val_acc: 0.8743
Epoch 7/100
46494/46494 [==============================] - 13s 283us/step - loss: 0.1723 - acc: 0.9447 - val_lo

Epoch 59/100
46494/46494 [==============================] - 13s 285us/step - loss: 0.0636 - acc: 0.9826 - val_loss: 0.8874 - val_acc: 0.8713
Epoch 60/100
46494/46494 [==============================] - 13s 284us/step - loss: 0.0669 - acc: 0.9810 - val_loss: 0.9298 - val_acc: 0.8819
Epoch 61/100
46494/46494 [==============================] - 13s 284us/step - loss: 0.0689 - acc: 0.9804 - val_loss: 0.8818 - val_acc: 0.8785
Epoch 62/100
46494/46494 [==============================] - 13s 285us/step - loss: 0.0649 - acc: 0.9823 - val_loss: 0.9071 - val_acc: 0.8795
Epoch 63/100
46494/46494 [==============================] - 13s 286us/step - loss: 0.0634 - acc: 0.9823 - val_loss: 0.8816 - val_acc: 0.8822
Epoch 64/100
46494/46494 [==============================] - 13s 286us/step - loss: 0.0631 - acc: 0.9824 - val_loss: 0.8651 - val_acc: 0.8826
Epoch 65/100
46494/46494 [==============================] - 13s 286us/step - loss: 0.0622 - acc: 0.9825 - val_loss: 0.9344 - val_acc: 0.8733
Epoch 66/100


In [73]:
predictions = model.predict(reshaped_test)

In [74]:
# F1-score measure
from sklearn.metrics import f1_score
num_classes = 18
class_predictions = []
class_true = []
tot_labels = 0.0
count = 0.0
for pair in zip(predictions, test_labels):
    class_predictions.append(np.argmax(pair[0]))
    class_true.append(np.argmax(pair[1]))
    if np.argmax(pair[0]) == np.argmax(pair[1]):
        count += 1.0
    tot_labels += 1.0
    
print('Standard accuracy is ' + str(count/tot_labels))    

unique, counts = np.unique(class_true, return_counts=True)
counted_labels = dict(zip(unique, counts))
f1_scores = f1_score(class_predictions, class_true, average=None)

tot_f1_score = 0.0
weights_sum = 0.0
for i in range(num_classes):
    labels_class_i = counted_labels[i]
    weight_i = labels_class_i / tot_labels
    weights_sum += weight_i
    tot_f1_score += f1_scores[i]*weight_i
    print(str(i) + ' ' + str(weight_i) + ' ' + str(f1_scores[i]))

    
print('The weigths sum is ' + str(weights_sum))
print('The computed f1-score is {}'.format(tot_f1_score))
print('The f1-score with sklearn function is {}'.format(f1_score(class_true, class_predictions, average='weighted')))

print('Average f1-score is {}'.format(np.mean(f1_scores)))

Standard accuracy is 0.87689508793208
0 0.832524762482 0.940791467184
1 0.0058621386699 0.622222222222
2 0.00960177885587 0.845360824742
3 0.00606428138266 0.661157024793
4 0.00838892257934 0.887573964497
5 0.0230442692541 0.579634464752
6 0.0161714170204 0.71186440678
7 0.0101071356378 0.491071428571
8 0.00778249444108 0.323699421965
9 0.00394178289873 0.423529411765
10 0.00424499696786 0.21686746988
11 0.0040428542551 0.1875
12 0.00262785526582 0.181818181818
13 0.0067717808773 0.460317460317
14 0.00616535273903 0.440677966102
15 0.0100060642814 0.515337423313
16 0.0320396199717 0.596330275229
17 0.0106124924196 0.444444444444
The weigths sum is 1.0
The computed f1-score is 0.8774429202028305
The f1-score with sklearn function is 0.8774429202028303
Average f1-score is 0.5294554365764078


In [75]:
from keras.models import Model
from sklearn.decomposition import PCA

layer_name = 'dense2_layer'
inter_layer_model = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)

network_output = inter_layer_model.predict(all_train)

# PCA with certain number of principal components
pca = PCA(n_components=12)
pca.fit(network_output)
new_output = pca.transform(network_output)

In [164]:
from sklearn import svm

svm_labels = np.argmax(all_labels, axis=1)

# train a one-vs-one multi-class support vector machine
clf = svm.SVC(decision_function_shape='ovo')
clf.fit(new_output, svm_labels)

# predict test data
svm_pred = clf.predict(pca.transform(inter_layer_model.predict(reshaped_test)))

# measure accuracy and f1-score
num = 0.0
den = 0.0
new_test_labels = np.argmax(test_labels, axis=1)
for pair in zip(svm_pred, new_test_labels):
    if pair[0] == pair[1]:
        num += 1.0
    
    den += 1.0
    
print('Test accuracy is: {}'.format(num / den))
f1_scores = f1_score(svm_pred, new_test_labels, average=None)
print('The f1-score with sklearn function is {}'.format(f1_score(new_test_labels, svm_pred, average='weighted')))
print('Average f1-score is {}'.format(np.mean(f1_scores)))

PCA(copy=True, iterated_power='auto', n_components=12, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

Test accuracy is: 0.8803315140489185
The f1-score with sklearn function is 0.8793259574667726
Average f1-score is 0.5377435802985444
